##Кредитный скоринг
###Определение одобренных заявок среди полных на базе обзвона.
####Подход "простыня" с Я.Каталогом

In [1]:
##### Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import hashlib
from collections import Counter

hive_config_query = '''
set hive.vectorized.execution.enabled=true;
set hive.vectorized.execution.reduce.enabled = true;
set mapreduce.map.memory.mb=4096;
set mapreduce.map.child.java.opts=-Xmx4g;
set mapreduce.task.io.sort.mb=1024;
set mapreduce.reduce.child.java.opts=-Xmx4g;
set mapreduce.reduce.memory.mb=7000;
set mapreduce.reduce.shuffle.input.buffer.percent=0.5;
set mapreduce.input.fileinputformat.split.minsize=536870912;
set mapreduce.input.fileinputformat.split.maxsize=1073741824;
set hive.optimize.ppd=true;
set hive.merge.smallfiles.avgsize=536870912;
set hive.merge.mapredfiles=true;
set hive.merge.mapfiles=true;
set hive.hadoop.supports.splittable.combineinputformat=true;
set hive.exec.reducers.bytes.per.reducer=536870912;
set hive.exec.parallel=true;
set hive.exec.max.created.files=10000000;
set hive.exec.compress.output=true;
set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.max.dynamic.partitions=1000000;
set hive.exec.max.dynamic.partitions.pernode=100000;
set io.seqfile.compression.type=BLOCK;
set mapreduce.map.failures.maxpercent=5;
'''

sc.stop()
conf = (SparkConf()
        .set("spark.executor.instances", 6)
        .set("spark.driver.maxResultSize", "16g")
        .set('spark.driver.memory','16g')
        .set("spark.executor.memory", '4g')
        .set("spark.yarn.executor.memoryOverhead", 1048)        
       )
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

for q in hive_config_query.split(';'):
    try:
        hc.sql(q)
    except:
        pass

In [2]:
df_all = (hc.sql('select * from user_kposminin.ccall_scoring_train_3')
        .toPandas()
         )

In [3]:
df_all.shape

NameError: name 'df_all' is not defined

In [2]:
df_all.show()

NameError: name 'df_all' is not defined

In [18]:
df_train  = df_all.query("call_ymd < '2016-11-15'")
df_valid  = df_all.query("'2016-11-15' <=  call_ymd < '2016-12-10'")
df_test   = df_all.query("call_ymd >= '2016-12-10'")
feat_cols = df_train.columns[3:-1]
label     = 'approve'

In [ ]:
import sklearn
feat_cols = df_train.columns[3:-1]
print("Features as classifiers: AUC ROC on test. ")
features_roc_auc = [(
        f,
        sklearn.metrics.roc_auc_score(y_true = df_train['approve'],y_score = df_train[f].fillna(-1000)),
        sklearn.metrics.roc_auc_score(y_true = df_valid['approve'],y_score = df_valid[f].fillna(-1000)),
    ) for f in feat_cols]

In [30]:
print('Feature importances\n{:<30}  {:<15}  {:<15}\n'.format('Feature','Train AUC ROC','Test AUC ROC') +     
    '\n'.join('{:<30}  {:<15.4f}  {:<15.4f}'.format(*e) for e in sorted(features_roc_auc, key = lambda e: -e[1])))

Feature importances
Feature                         Train AUC ROC    Test AUC ROC   
avg_score1                      0.6129           0.6434         
q30_score1                      0.6053           0.6389         
q50_score1                      0.6039           0.6371         
q70_score1                      0.5999           0.6255         
q90_score1                      0.5717           0.5868         
good_urlfr_share_score1         0.5562           0.5681         
q95_score1                      0.5532           0.5612         
avg_hour_q10                    0.5398           0.5421         
avg_hour                        0.5368           0.5399         
q30_score2                      0.5365           0.5555         
avg_hour_q90                    0.5342           0.5379         
max_score1                      0.5319           0.5408         
q50_score2                      0.5257           0.5438         
avg_time_std                    0.5249           0.5308         
avg_y

In [39]:
import xgboost as xgb
param = {}
#param = {'bst:max_depth':8, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic', 'alpha':1, 'tree_method':'approx'}
param['eval_metric'] = ['auc','logloss']

dtrain = xgb.DMatrix( df_train[feat_cols], label=df_train['approve'],missing=np.nan)
dvalid = xgb.DMatrix( df_valid[feat_cols], label=df_valid['approve'],missing=np.nan)

evallist  = [(dvalid,'eval'), (dtrain,'train')]
num_round = 100
bst = xgb.train( param, dtrain, num_round, evallist)

[0]	eval-auc:0.657841	eval-logloss:0.668191	train-auc:0.645628	train-logloss:0.666246
[1]	eval-auc:0.663980	eval-logloss:0.654280	train-auc:0.653429	train-logloss:0.651358
[2]	eval-auc:0.665860	eval-logloss:0.646290	train-auc:0.657939	train-logloss:0.642585
[3]	eval-auc:0.667650	eval-logloss:0.640722	train-auc:0.660919	train-logloss:0.637224
[4]	eval-auc:0.669597	eval-logloss:0.636873	train-auc:0.664309	train-logloss:0.633403
[5]	eval-auc:0.670080	eval-logloss:0.634456	train-auc:0.667450	train-logloss:0.630574
[6]	eval-auc:0.671046	eval-logloss:0.632597	train-auc:0.669779	train-logloss:0.628483
[7]	eval-auc:0.671502	eval-logloss:0.631535	train-auc:0.672134	train-logloss:0.626701
[8]	eval-auc:0.671970	eval-logloss:0.630541	train-auc:0.674187	train-logloss:0.625130
[9]	eval-auc:0.672098	eval-logloss:0.630075	train-auc:0.676058	train-logloss:0.623861
[10]	eval-auc:0.672099	eval-logloss:0.629603	train-auc:0.677883	train-logloss:0.622657
[11]	eval-auc:0.671895	eval-logloss:0.629597	train-au

лучшая итерация [12]	eval-auc:0.672253	eval-logloss:0.629216	train-auc:0.681619	train-logloss:0.620445

###Яндекс каталог.

In [40]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=True)

df_train_yaca_sparse = v.fit_transform(df_train['yaca_str'].map(lambda s: { kv.split(':')[0]:float(kv.split(':')[1])  for kv in s.split(' ')} if len(s) > 0 else {}))
df_valid_yaca_sparse = v.transform(df_valid['yaca_str'].map(lambda s: { kv.split(':')[0]:float(kv.split(':')[1])  for kv in s.split(' ')} if len(s) > 0 else {}))

In [41]:
import sklearn.linear_model
clfLR = sklearn.linear_model.LogisticRegression(penalty = 'l1')

clfLR.fit(df_train_yaca_sparse,df_train['approve'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [42]:
print('Logistic regression for yaca features on train {:.4f}; test: {:.4f}'.format(
           sklearn.metrics.roc_auc_score(y_true = df_train['approve'],y_score = clfLR.predict_proba(df_train_yaca_sparse)[:,1]),
           sklearn.metrics.roc_auc_score(y_true = df_valid['approve'],y_score = clfLR.predict_proba(df_valid_yaca_sparse)[:,1])
    ))

Logistic regression for yaca features on train 0.5975; test: 0.6131


In [43]:
import sklearn.ensemble
clfRF = sklearn.ensemble.RandomForestClassifier(max_depth = 5,n_estimators = 50)

clfRF.fit(df_train_yaca_sparse,df_train['approve'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [44]:
print('Random Forest for yaca features on train {:.4f}; test: {:.4f}'.format(
           sklearn.metrics.roc_auc_score(y_true = df_train['approve'],y_score = clfRF.predict_proba(df_train_yaca_sparse)[:,1]),
           sklearn.metrics.roc_auc_score(y_true = df_valid['approve'],y_score = clfRF.predict_proba(df_valid_yaca_sparse)[:,1])
    ))

Random Forest for yaca features on train 0.6055; test: 0.6120


In [46]:
#xgboost for yaca stangalone
dtrain_yaca = xgb.DMatrix( df_train_yaca_sparse, label=df_train['approve'])
dvalid_yaca = xgb.DMatrix( df_valid_yaca_sparse, label=df_valid['approve'])

evallist  = [(dvalid_yaca,'eval'), (dtrain_yaca,'train')]
num_round = 40
bst = xgb.train( param, dtrain_yaca, num_round, evallist )

[0]	eval-auc:0.593922	eval-logloss:0.674753	train-auc:0.584284	train-logloss:0.675300
[1]	eval-auc:0.607281	eval-logloss:0.664229	train-auc:0.597525	train-logloss:0.665355
[2]	eval-auc:0.612022	eval-logloss:0.658553	train-auc:0.603809	train-logloss:0.659550
[3]	eval-auc:0.614250	eval-logloss:0.655283	train-auc:0.609622	train-logloss:0.655761
[4]	eval-auc:0.618075	eval-logloss:0.652824	train-auc:0.614399	train-logloss:0.653013
[5]	eval-auc:0.619228	eval-logloss:0.651113	train-auc:0.617438	train-logloss:0.650917
[6]	eval-auc:0.620353	eval-logloss:0.650000	train-auc:0.620854	train-logloss:0.649090
[7]	eval-auc:0.621623	eval-logloss:0.648941	train-auc:0.624441	train-logloss:0.647406
[8]	eval-auc:0.623245	eval-logloss:0.648205	train-auc:0.627016	train-logloss:0.646141
[9]	eval-auc:0.623943	eval-logloss:0.647595	train-auc:0.630074	train-logloss:0.644810
[10]	eval-auc:0.626174	eval-logloss:0.646944	train-auc:0.632759	train-logloss:0.643615
[11]	eval-auc:0.626588	eval-logloss:0.646658	train-au

лучшая итерация [37]	eval-auc:0.631754	eval-logloss:0.643753	train-auc:0.663964	train-logloss:0.628422

In [70]:
#bst = xgb.train( param, dtrain_yaca, 40, evallist)

df_train.loc[:,'yaca_pred'] = bst.predict(dtrain_yaca)
df_valid.loc[:,'yaca_pred'] = bst.predict(dvalid_yaca)

/opt/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [80]:
#param = {'bst:max_depth':4, 'bst:eta':0.1, 'silent':1, 'objective':'binary:logistic', 'alpha':0.5, 'tree_method':'approx'}
param['eval_metric'] = ['auc','logloss']

#dtrain = xgb.DMatrix(df_train[feat_cols.tolist() + ['yaca_pred']], label=df_train['approve'],missing=np.nan)
#dvalid = xgb.DMatrix(df_valid[feat_cols.tolist() + ['yaca_pred']], label=df_valid['approve'],missing=np.nan)

evallist  = [(dvalid,'eval'), (dtrain,'train')]
num_round = 400
bst = xgb.train( param, dtrain, num_round, evallist )

[0]	eval-auc:0.639745	eval-logloss:0.682946	train-auc:0.665246	train-logloss:0.680305
[1]	eval-auc:0.643210	eval-logloss:0.674641	train-auc:0.672546	train-logloss:0.669756
[2]	eval-auc:0.643464	eval-logloss:0.668162	train-auc:0.673452	train-logloss:0.661082
[3]	eval-auc:0.644969	eval-logloss:0.662715	train-auc:0.674166	train-logloss:0.653913
[4]	eval-auc:0.646180	eval-logloss:0.658272	train-auc:0.676630	train-logloss:0.647878
[5]	eval-auc:0.646872	eval-logloss:0.654583	train-auc:0.677509	train-logloss:0.642705
[6]	eval-auc:0.649759	eval-logloss:0.651676	train-auc:0.678514	train-logloss:0.638559
[7]	eval-auc:0.649614	eval-logloss:0.649285	train-auc:0.679001	train-logloss:0.634919
[8]	eval-auc:0.649669	eval-logloss:0.647325	train-auc:0.679767	train-logloss:0.631801
[9]	eval-auc:0.650104	eval-logloss:0.645837	train-auc:0.680237	train-logloss:0.629238
[10]	eval-auc:0.650459	eval-logloss:0.644560	train-auc:0.680904	train-logloss:0.627008
[11]	eval-auc:0.650476	eval-logloss:0.643678	train-au

####Использование предсказания yaca в качесте фактора не дает улучшения, даже при том, что мы переобучаемся на валидационной выборке.

In [81]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=True)

df_train_yaca_sparse1 = v.fit_transform(df_train['yaca_str'].map(lambda s: { kv.split(':')[0]:1  for kv in s.split(' ')} if len(s) > 0 else {}))
df_valid_yaca_sparse1 = v.transform(df_valid['yaca_str'].map(lambda s: { kv.split(':')[0]:1  for kv in s.split(' ')} if len(s) > 0 else {}))

In [84]:
#xgboost for yaca stangalone with 1
dtrain_yaca_1 = xgb.DMatrix( df_train_yaca_sparse1, label=df_train['approve'])
dvalid_yaca_1 = xgb.DMatrix( df_valid_yaca_sparse1, label=df_valid['approve'])

evallist  = [(dvalid_yaca_1,'eval'), (dtrain_yaca_1,'train')]
num_round = 800
bst = xgb.train( param, dtrain_yaca_1, num_round, evallist )

[0]	eval-auc:0.573171	eval-logloss:0.686954	train-auc:0.563877	train-logloss:0.687275
[1]	eval-auc:0.573903	eval-logloss:0.681915	train-auc:0.565460	train-logloss:0.682618
[2]	eval-auc:0.577597	eval-logloss:0.677795	train-auc:0.568859	train-logloss:0.678589
[3]	eval-auc:0.577899	eval-logloss:0.674357	train-auc:0.569310	train-logloss:0.675438
[4]	eval-auc:0.584798	eval-logloss:0.671527	train-auc:0.574445	train-logloss:0.672732
[5]	eval-auc:0.587507	eval-logloss:0.669167	train-auc:0.577016	train-logloss:0.670465
[6]	eval-auc:0.589388	eval-logloss:0.667153	train-auc:0.578691	train-logloss:0.668571
[7]	eval-auc:0.590974	eval-logloss:0.665481	train-auc:0.580820	train-logloss:0.666969
[8]	eval-auc:0.594633	eval-logloss:0.663952	train-auc:0.584145	train-logloss:0.665663
[9]	eval-auc:0.595703	eval-logloss:0.662739	train-auc:0.584972	train-logloss:0.664490
[10]	eval-auc:0.596225	eval-logloss:0.661717	train-auc:0.585613	train-logloss:0.663548
[11]	eval-auc:0.597113	eval-logloss:0.660806	train-au

####Удивительно, но с 1 в значениях получилось хуже, чем с TF.          
####TODO: попробовать TFIDF

#####все факторы yaca + простыней

In [85]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)

df_train_yaca_dense = v.fit_transform(df_train['yaca_str'].map(lambda s: { kv.split(':')[0]:float(kv.split(':')[1])  for kv in s.split(' ')} if len(s) > 0 else {}))
df_valid_yaca_dense = v.transform(df_valid['yaca_str'].map(lambda s: { kv.split(':')[0]:float(kv.split(':')[1])  for kv in s.split(' ')} if len(s) > 0 else {}))
#df_train_yaca_dense = v.fit_transform(df_train['yaca_str'].map(lambda s: { kv.split(':')[0]:1  for kv in s.split(' ')} if len(s) > 0 else {}))
#df_test_yaca_dense = v.transform(df_test['yaca_str'].map(lambda s: { kv.split(':')[0]:1 for kv in s.split(' ')} if len(s) > 0 else {}))

In [86]:
yaca_cols = ['yaca_{}'.format(i) for i in range(df_train_yaca_dense.shape[1])]
for i in range(df_train_yaca_dense.shape[1]):
    df_train.loc[:,'yaca_{}'.format(i)] = df_train_yaca_dense[:,i]
    df_valid.loc[:,'yaca_{}'.format(i)] = df_valid_yaca_dense[:,i]

/opt/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [88]:
feat_cols_w_yaca = feat_cols.tolist() + yaca_cols
dtrain_all_yaca = xgb.DMatrix( df_train[feat_cols_w_yaca], label=df_train['approve'], missing = np.nan)
dvalid_all_yaca = xgb.DMatrix( df_valid[feat_cols_w_yaca], label=df_valid['approve'], missing = np.nan)

In [105]:

param = {
    'bst:max_depth': 5,     
    'silent': 1, 
    'objective':'binary:logistic', 
    'alpha': 1, 
    'tree_method':'approx',
    'learning_rate': 0.04,
    'eval_metric' : ['logloss','auc']
}
#'bst:eta':1, 
#    'tree_method':'approx',
evallist  = [(dvalid_all_yaca,'eval'), (dtrain_all_yaca,'train')]
num_round = 730
bst = xgb.train( param, dtrain_all_yaca, num_round, evallist,early_stopping_rounds = 10)
print(bst.best_iteration)

Will train until train error hasn't decreased in 10 rounds.
Multiple eval metrics have been passed: 'auc' will be used for early stopping.

[0]	eval-logloss:0.689497	eval-auc:0.657501	train-logloss:0.689237	train-auc:0.640511
[1]	eval-logloss:0.686141	eval-auc:0.662027	train-logloss:0.685583	train-auc:0.645628
[2]	eval-logloss:0.683014	eval-auc:0.663051	train-logloss:0.682185	train-auc:0.646329
[3]	eval-logloss:0.680125	eval-auc:0.663866	train-logloss:0.679047	train-auc:0.647185
[4]	eval-logloss:0.677381	eval-auc:0.664163	train-logloss:0.676154	train-auc:0.647628
[5]	eval-logloss:0.674875	eval-auc:0.663847	train-logloss:0.673454	train-auc:0.647594
[6]	eval-logloss:0.672478	eval-auc:0.664598	train-logloss:0.670935	train-auc:0.648208
[7]	eval-logloss:0.670292	eval-auc:0.664582	train-logloss:0.668596	train-auc:0.648597
[8]	eval-logloss:0.668189	eval-auc:0.664539	train-logloss:0.666414	train-auc:0.648894
[9]	eval-logloss:0.666243	eval-auc:0.664769	train-logloss:0.664384	train-auc:0.649114


729


[729]	eval-logloss:0.623221	eval-auc:0.680688	train-logloss:0.601157	train-auc:0.716410


In [ ]:
параметры по умолчанию [401]	eval-auc:0.680291	eval-logloss:0.623436	train-auc:0.703328	train-logloss:0.608144
                
------                 
param = {
    'bst:max_depth': 5,     
    'silent': 1, 
    'objective':'binary:logistic', 
    'alpha': 1, 
    'tree_method':'approx',
    'learning_rate': 0.04,
    'eval_metric' : ['logloss','auc']
}
[733]	eval-logloss	0.623207	eval-auc	0.680725	train-logloss	0.601083	train-auc	0.716556

                
----------


In [106]:
#df_test preparation
df_test_yaca_dense = v.transform(df_test['yaca_str'].map(lambda s: { kv.split(':')[0]:float(kv.split(':')[1])  for kv in s.split(' ')} if len(s) > 0 else {}))
for i in range(df_train_yaca_dense.shape[1]):
    df_test.loc[:,'yaca_{}'.format(i)] = df_test_yaca_dense[:,i]
dtest_all_yaca = xgb.DMatrix( df_test[feat_cols_w_yaca], label=df_test['approve'], missing = np.nan)

In [114]:
df_train.loc[:,'pred'] = bst.predict(dtrain_all_yaca)
df_valid.loc[:,'pred'] = bst.predict(dvalid_all_yaca)
df_test.loc[:,'pred'] = bst.predict(dtest_all_yaca)

In [111]:
def metrics(y_true,y_score,lift = None, return_str = False):
    import sklearn
    import collections
    
    if True:
        res = collections.OrderedDict()
        samp_size = len(y_true)
        res['Sample size'] = samp_size
        res['Posit share'] = sum(y_true) * 1./ samp_size
        res['Sample size'] = len(y_true)
        res['AUC ROC'] = sklearn.metrics.roc_auc_score(y_true = y_true, y_score = y_score)
        res['AUC PR'] = sklearn.metrics.auc(
                        *sklearn.metrics.precision_recall_curve(y_true = y_true, probas_pred  = y_score)[:2],
                        reorder = True
        )
        res['Log loss'] = sklearn.metrics.log_loss(y_true = y_true, y_pred = y_score)
        if lift:
            predictions_and_labels = sorted(zip(y_score,y_true), reverse = True)
            for l in lift:
                res['Lift ' + str(l)] = sum([e[1] for e in predictions_and_labels[:int(l * samp_size)]]) * 1. / int(l * samp_size) / res['Posit share']
                
        if return_str:
            res = '\n'.join(['{:<12}: {:.5f}'.format(k,v) for (k,v) in res.items()]) + '.'
        return res

In [137]:
print('Feature importances:\n' + '\n'.join('{:<25}  {:<5}'.format(k,v) for k,v in sorted(bst.get_fscore().items(),key = lambda e:-e[1])))

Feature importances:
avg_duration               536  
avg_score1                 515  
social_share               502  
max_score3                 492  
hour_std                   470  
hits                       435  
avg_ymd_cnt                419  
q95_score3                 414  
avg_time_std               405  
max_score2                 358  
mobile_share               357  
cnt                        352  
q50_score1                 346  
avg_hour_q10               339  
q70_score2                 339  
q70_score1                 338  
avg_score3                 330  
q30_score1                 325  
yaca_128                   325  
q30_score3                 319  
work_hours_hits_share      318  
q30_score2                 293  
avg_ymd_range              288  
q50_score2                 275  
q90_score1                 263  
q90_score2                 261  
visits_cnt                 247  
avg_hour                   234  
avg_hour_q90               233  
avg_score2            

In [120]:
print('All features + all yaca data xgboost model.\n\nTrain \n{}\n\nValid\n{}\n\nTest\n{}\n'.format(
        metrics(y_true = df_train['approve'],y_score = df_train['pred'],lift = [0.05,0.1,0.2,0.5],return_str = True),
        metrics(y_true = df_valid['approve'],y_score = df_valid['pred'],lift = [0.05,0.1,0.2,0.5],return_str = True),
        metrics(y_true = df_test['approve'],y_score = df_test['pred'],lift = [0.05,0.1,0.2,0.5],return_str = True)
    ))

All features + all yaca data xgboost model.

Train 
Sample size : 193415.00000
Posit share : 0.39659
AUC ROC     : 0.71641
AUC PR      : 0.22373
Log loss    : 0.60116
Lift 0.05   : 2.01721
Lift 0.1    : 1.85649
Lift 0.2    : 1.66708
Lift 0.5    : 1.37223.

Valid
Sample size : 32312.00000
Posit share : 0.39552
AUC ROC     : 0.68069
AUC PR      : 0.16502
Log loss    : 0.62322
Lift 0.05   : 1.73773
Lift 0.1    : 1.63782
Lift 0.2    : 1.52631
Lift 0.5    : 1.31002.

Test
Sample size : 23463.00000
Posit share : 0.39138
AUC ROC     : 0.68040
AUC PR      : 0.36264
Log loss    : 0.62392
Lift 0.05   : 1.72733
Lift 0.1    : 1.65762
Lift 0.2    : 1.54054
Lift 0.5    : 1.31074.



### Результат лучше всех предыдущих, но в целом качество еще оставляет сомненения, можно ли это применять на проде.